## try different feature selection

In [2]:
import pandas as pd
df= pd.read_csv('churn_prediction_data/churn_prediction_data/train.csv')

##alter the categorial data to numerical type 
df_dummies = pd.get_dummies(df, columns=['Geography','Gender'], dtype=float)

##alter some extreme number by feature scaling
minmax_col = ['CreditScore', 'Balance', 'EstimatedSalary']
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_dummies[minmax_col] = scaler.fit_transform(df_dummies[minmax_col])

df_dummies["Age"]=pd.to_numeric(df_dummies["Age"],downcast="float")
df_dummies["Tenure"]=pd.to_numeric(df_dummies["Tenure"],downcast="float")
df_dummies["NumOfProducts"]=pd.to_numeric(df_dummies["NumOfProducts"],downcast="float")
df_dummies["IsActiveMember"]=pd.to_numeric(df_dummies["IsActiveMember"],downcast="float")

df_dummies

,RowNumber,CustomerId,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,551,15806307,Trevisano,0.892508,38.0,5.0,0.409621,2.0,0,1.0,0.589163,0,0.0,0.0,1.0,0.0,1.0
1,6897,15709621,Martin,0.768730,54.0,4.0,0.000000,1.0,1,0.0,0.654032,1,1.0,0.0,0.0,1.0,0.0
2,4588,15619340,Palmer,0.736156,31.0,5.0,0.456025,1.0,1,1.0,0.887514,0,1.0,0.0,0.0,1.0,0.0
3,291,15620746,Napolitani,0.475570,40.0,4.0,0.331954,1.0,1,0.0,0.627770,0,0.0,0.0,1.0,1.0,0.0
4,1673,15646372,Yao,1.000000,42.0,5.0,0.459864,1.0,0,1.0,0.810459,0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,5345,15584532,Yu,0.397394,35.0,6.0,0.465350,2.0,1,1.0,0.792944,1,1.0,0.0,0.0,1.0,0.0
7996,5837,15606641,Liao,0.508143,45.0,7.0,0.661749,1.0,1,0.0,0.491455,0,0.0,1.0,0.0,1.0,0.0
7997,7335,15739692,Ferri,0.758958,43.0,5.0,0.558373,1.0,1,0.0,0.862275,1,0.0,0.0,1.0,1.0,0.0
7998,9552,15791373,Worsnop,0.876221,38.0,4.0,0.446714,1.0,0,0.0,0.446828,0,1.0,0.0,0.0,0.0,1.0


In [3]:
all_inputs = df_dummies[['CreditScore','Age','Tenure','Balance','NumOfProducts',
                 'IsActiveMember','EstimatedSalary','Geography_France','Geography_Germany','Geography_Spain',
                 'Gender_Female','Gender_Male']].values
all_classes = df_dummies['Exited'].values

print('before',all_inputs.shape)

before (8000, 12)


In [4]:
from sklearn.ensemble import RandomForestRegressor
# create the random forest with your hyperparameters.
model = RandomForestRegressor(n_estimators=340)
# fit the model to start training.
model.fit(all_inputs,all_classes)
# get the importance of the resulting features.
importances = model.feature_importances_
# create a data frame for visualization.
final_df = pd.DataFrame({"Importances":importances})
final_df.set_index('Importances')
all_inputs=all_inputs * importances

## [怎麼做一個好的特徵選擇](https://medium.com/jackys-blog/%E6%80%8E%E9%BA%BC%E5%81%9A%E4%B8%80%E5%80%8B%E5%A5%BD%E7%9A%84%E7%89%B9%E5%BE%B5%E9%81%B8%E6%93%87-%E4%BD%BF%E7%94%A8python-part-1-d698cc15d76e)
過濾法(FIlter)中的Univariate feature selection(單變量特徵選擇)單獨計算每個feature的統計指標
,根據該指標判斷哪些指標重要,剔除那些不重要的指標
(1) classification(分類): 使用卡方檢定

In [5]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
#只選得分最好的k＝11個features.
all_inputs=SelectKBest(chi2,k=11).fit_transform(all_inputs,all_classes)
print('after',all_inputs.shape)

after (8000, 11)


In [6]:
from sklearn.model_selection import train_test_split
(x_train,x_test,y_train,y_test) = train_test_split(all_inputs, all_classes, train_size=0.80, random_state=1,stratify=all_classes)


In [7]:
from xgboost.sklearn import XGBClassifier
# 建立 XGBClassifier 模型
xgboostModel = XGBClassifier(n_estimators=1000, learning_rate= 0.5, use_label_encoder=False)
# 使用訓練資料訓練模型
xgboostModel.fit(x_train,y_train)
# 使用訓練資料預測分類
predicted = xgboostModel.predict(x_train)

# 預測成功的比例
print('訓練集: ',xgboostModel.score(x_train,y_train))
print('測試集: ',xgboostModel.score(x_test, y_test))

[01:59:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
訓練集:  1.0
測試集:  0.84


In [8]:
from sklearn.neural_network import MLPClassifier
classifier = MLPClassifier(hidden_layer_sizes=(1152,576,256,128,64,32),activation='relu',random_state=1)
classifier.fit(x_train,y_train)
classifier.score(x_test, y_test)

0.864375

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import accuracy_score
#使用網格搜索確定要建立的基學習器個數
clf = GridSearchCV(RandomForestClassifier(max_features="log2"),param_grid=({"n_estimators":range(1,101,10)}),cv=10)
clf.fit(x_train,y_train)
print(clf.best_params_)
#再使用網格搜索來確定決策樹的參數
clf2 = GridSearchCV(RandomForestClassifier(n_estimators=11),param_grid=({"max_depth":range(1,10)}))
clf2.fit(x_train,y_train)
print(clf2.best_params_)
#根據最大層數3，最多棵樹11，建立最終的隨機森林來預測
rf = RandomForestClassifier(n_estimators=11,max_depth=10,max_features="log2")
rf.fit(x_train,y_train)
y_hat = rf.predict(x_test)
print(accuracy_score(y_hat,y_test))

{'n_estimators': 91}
{'max_depth': 8}
0.858125


In [10]:
from sklearn.ensemble import RandomForestClassifier
#after ajusting data, begin to use it in algorithm to predict the result
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_jobs=-1,oob_score=True,  random_state=2,max_depth=10)
classifier.fit(x_train,y_train)

#adjusting the testing data and then use in algorithm  
#test_inputs = pca.transform(test_inputs)
y_pred = classifier.predict(x_test) 
classifier.score(x_test, y_test)

0.865625